In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import io
from google.colab import drive
import threading
import multiprocessing
import time
import tensorflow as tf
drive.mount('/content/drive')
import tensorflow as tf
tf.test.gpu_device_name()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import glob
glob.glob("/content/drive/My Drive/google_notebook/M5_traindata2/*")

['/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/calendar.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sample_submission.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a']

In [2]:
import os.path

train_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv")
calendar_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar.csv")
sellprices2_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv")
calendar_data2 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv")

days_num = 1913

cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "wm_yr_wk", "wday", "month", "year", "d",
        "Religious", "National", "Cultural", "Sporting", "snap"] #"price", "price_down", "price_up", "value"は後程追加
train_data2a0 = pd.DataFrame(index=[ij for ij in range(days_num)], columns=cols)

start_n = 0

thread_num = 2 #スレッド数
process_num = 8 #プロセス数

print(len(train_data))

current_num = [-1 for jj in range(thread_num + process_num)]  # 実行中のファイル番号(重複処理を防ぐ)

def stap(ST):
        return calendar_data2["snap_" + ST]

def traindata2_making(th_num):
    for ii in range(start_n, len(train_data)):
        if ((ii in current_num) == False): #その行は処理されていない
            current_num[th_num] = ii
            if (os.path.isfile("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv") == False):  # ファイルがまだ作成されていない
                train_data2a = train_data2a0.copy()
                tra_sir = train_data.iloc[ii]
                for jj in range(days_num):train_data2a.iloc[jj,0:6] = tra_sir[0:6]
                train_data2a.iloc[:,6] = calendar_data.iloc[:,1] #週
                train_data2a.iloc[:,7:11] = calendar_data.iloc[:,3:7] #年, 月, 曜日
                train_data2a.iloc[:,11:15] = calendar_data2.iloc[:,1:5] #イベント情報
                train_data2a.iloc[:,15] = stap(tra_sir[5]) #snap

                sap_mindata = sellprices2_data[sellprices2_data.item_id == tra_sir[1]]  # sellpricesの中から必要なデータを抽出
                sap_mindata = sap_mindata[sap_mindata.store_id == tra_sir[4]]
                sap_mindata = sap_mindata.drop(["store_id", "item_id"], axis=1)

                train_data2a = pd.merge(train_data2a, sap_mindata, on='wm_yr_wk', how='left') #価格とその変化

                train_data2a["value"] = tra_sir[6:].to_list()

                train_data2a.to_csv(
                    "/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv", index=False)
            current_num[th_num] = -1

thread_list = []
with tf.device('/cpu:0'):
  for i in range(thread_num):
      thread = threading.Thread(target=traindata2_making, args=(i,))
      thread.start()
      thread_list.append(thread)
      sleep_seconds = 1
      time.sleep(sleep_seconds)

print("aaaaaaaaaaaaaa")

with tf.device('/gpu:0'):
  for i2 in range(thread_num, thread_num + process_num):
      mp = multiprocessing.Process(target=traindata2_making, args=(i2,))
      mp.start()
      print(i2)
      sleep_seconds = 1
      time.sleep(sleep_seconds)

while(True):
  print('\r', end='')
  print(current_num, end='')
  sleep_seconds = 2
  time.sleep(sleep_seconds)

for thread in thread_list:
    thread.join()

30490
aaaaaaaaaaaaaa
[3583, 3582, 3560, 3004, -1, -1, -1, -1, -1, -1, -1, -1]

Process Process-2:
Process Process-3:
Process Process-8:
Process Process-6:
Process Process-7:
Process Process-5:
Process Process-4:
Process Process-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/proc

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-83d665e87c80>", line 74, in <module>
    time.sleep(sleep_seconds)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 313,

KeyboardInterrupt: ignored

In [0]:
tra_sir = train_data.iloc[0]
tra_sir[6:]


d_1       0
d_2       0
d_3       0
d_4       0
d_5       0
         ..
d_1909    1
d_1910    3
d_1911    0
d_1912    1
d_1913    1
Name: 0, Length: 1913, dtype: object

In [0]:
cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "wm_yr_wk", "wday", "month", "year", "d",
        "Religious", "National", "Cultural", "Sporting", "snap", "price", "price_down", "price_up", "sold", "value"]
train_data2a0 = pd.DataFrame(index=[ij for ij in range(1913)], columns=cols)
train_data2a0

,id,item_id,dept_id,cat_id,store_id,state_id,wm_yr_wk,wday,month,year,d,Religious,National,Cultural,Sporting,snap,price,price_down,price_up,value
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
sellprices2_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv")
sap_mindata = sellprices2_data[sellprices2_data.item_id == "HOBBIES_1_001"]  # sellpricesの中から必要なデータを抽出
sap_mindata = sap_mindata[sap_mindata.store_id == "CA_1"]
sap_mindata

,store_id,item_id,wm_yr_wk,sell_price,price_down,price_up
0,CA_1,HOBBIES_1_001,11325,9.58,0,0
1,CA_1,HOBBIES_1_001,11326,9.58,1,0
2,CA_1,HOBBIES_1_001,11327,8.26,2,0
3,CA_1,HOBBIES_1_001,11328,8.26,0,0
4,CA_1,HOBBIES_1_001,11329,8.26,0,0
...,...,...,...,...,...,...
149,CA_1,HOBBIES_1_001,11617,8.38,0,0
150,CA_1,HOBBIES_1_001,11618,8.38,0,0
151,CA_1,HOBBIES_1_001,11619,8.38,0,0
152,CA_1,HOBBIES_1_001,11620,8.38,0,0


In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'